<a href="https://colab.research.google.com/github/IFibla/Aprenentatge-Automatic/blob/master/Problema%201/Individual-%C2%BFKullback-que%3F/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ¿Kullback que?

Cuando trabajamos con modelos que representan una distribución de probabilidad nuestro objetivo es hacer que la distribución de los datos se acerque lo más posible a las probabilidades que nos da el modelo sobre esos datos. Existen muchas maneras de calcular esa diferencia, una común es usar funciones de divergencia, entre ellas la divergencia de Kullback-Leibler es la más usada. Dadas dos distribuciones de probabilidad 𝑃 y 𝑄 se define asumiendo que sean distribuciones discretas como:

$$
KL(P|Q)=\sum_{i=1}^{N}P(i)\cdot \log(\frac{P(i)}{Q(i)}) 
$$

En el caso de distribuciones continuas, simplemente substituimos el sumatorio por una integral.



In [ ]:
%load_ext autoreload
%autoreload 2

!pip uninstall scikit-learn -y
!pip install -U scikit-learn

In [136]:
from jax import grad, jit
from sklearn.datasets import make_classification, make_circles
import jax.numpy as np
import matplotlib.pyplot as plt
import sklearn

In [71]:
RANDOM_STATE = 1

## Utils

In [22]:
def resize_make_classification_list(in_list):
  out_list = []
  for i in range(len(in_list[0])):
    out_list.append([l[i] for l in in_list])
  return out_list

In [49]:
def generate_matplotlib_histogram(X, labelX, labelY, title, rangeX=None, rangeY=None, bins=30):
  if rangeX is None: 
    rangeX = [np.min(X), np.max(X)]
  plt.hist(X, bins=bins, alpha=1, range=rangeX, density=True, stacked=True)  
  plt.xlabel(labelX)
  plt.ylabel(labelY)
  plt.title(title)
  if rangeY is not None:
    plt.ylim((rangeY[0], rangeY[1]))
  plt.legend([len(x) for x in X] if len(X) > 1 else "X")
  plt.show()

> Siendo 𝑋 una muestra de datos 𝑥1, ... , 𝑥𝑛 de valores discretos, donde podemos estimar su distribución 𝑃 a partir de su frecuencia y 𝑄 es una distribución de probabilidad sobre el mismo rango de valores discretos. Demuestra que optimizar 𝐾 𝐿(𝑃 |𝑄) es equivalente a optimizar la log verosimilitud negativa de 𝑄 sobre los datos.

In [126]:
a_minus_log_verosimilitud_q = lambda P: sum(1/np.log(P)) - (len(P) * np.log(len(P)))
a_diff_minus_log_verosimilitud_q = lambda P: sum(1/np.log(P))

a_optimise_kl = lambda P: (len(P) + len(P) * np.log(len(P))) * sum(P)
a_diff_optimise_kl = lambda P: (len(P) + len(P) * np.log(len(P))) * sum(P)

> Todo modelo de clasificación es una distribución de probabilidad sobre un conjunto de valores discretos, por lo que podemos ajustar un modelo probabilístico para clasificación haciendo que las probabilidades que obtenga para una muestra se ajusten a las de los datos. Usa la función make_classification de scikit-learn para crear un conjunto de datos de clasificación de dos dimensiones y 100 ejemplos. Tendrás dar un valor 0 al parámetro n_redundant y un valor 1 al parámetro n_clusters_per_class. Da un valor también al parámetro random_state para que los experimentos sean reproducibles. El problema que generará será de clasificación binaria.

[Documentation. make_classification](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html)


In [ ]:
b_sample = make_classification(n_samples=100, n_features=2, n_informative=2, n_redundant=0, n_clusters_per_class=1, random_state=RANDOM_STATE)
b_sample_prob = np.array(resize_make_classification_list(b_sample[0]))
b_sample_label = np.array(b_sample[1])
b_sample_prob, b_sample_label

> Podemos crear un modelo probabilístico con una función linear $f(w,x)=w\cdot x$. Para obtener probabilidades simplemente tenemos que aplicar sobre el resultado una función que de un valor entre 0 y 1. Por ejemplo la función sigmoide 𝜎:
>
> $σ(x)=\frac{1}{1+e^x}$
>
> A partir de la divergencia de Kullback-Leibler simplificando para problemas binarios podemos llegar a la función de pérdida de entropía cruzada binaria (binary cross entropy):
>
> $BCE(p(x), y )=y \cdot \log(p(x)) + (1-y) \cdot \log(1-p(x))$
>
> Donde 𝑝(𝑥) es la probabilidad que le asigna el modelo a un ejemplo, e 𝑦 es la etiqueta que le corresponde a los datos. Implementa un algoritmo de descenso de gradiente usando JAX con la función de entropía cruzada binaria. Explora diferentes tasas de aprendizaje. Comenta lo que observes en el comportamiento del error y los parámetros durante la optimización. Escoge un número de iteraciones y un valor para decidir el final de la optimización que te parezcan adecuados.

[Documentation. jax.grad](https://jax.readthedocs.io/en/latest/_autosummary/jax.grad.html)

In [145]:
c_sigmoid_function = lambda x: 1 / (1 + np.exp(x))
cross_binary_entropy = lambda Px, y: np.array(y * np.log(Px) + ((1 - y) * np.log(1-Px))).mean()

In [146]:
c_gradient_function = grad(cross_binary_entropy, argnums=1, allow_int=True)

In [147]:
c_sample_prob, c_sample_label = c_sigmoid_function(b_sample_prob), b_sample_prob

In [ ]:
c_gradient_function(c_sample_prob, c_sample_label)

> Genera un conjunto de datos con la función make_circles de scikit-learn usando el valor 0,1 para el parámetro noise. Optimiza el modelo para varios parámetros iniciales diferentes del modelo. Cuenta que esta que sucediendo e intenta explicar el porqué.

[Documentation. sklearn.datasets.make_circles](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_circles.html)

In [ ]:
d_sample = make_circles(n_samples=100, noise=0.1, random_state=RANDOM_STATE)
d_sample

> La función de entropía cruzada parece una función extraña para optimizar cuando lo que nos interesa es un modelo que tenga el mínimo número de ejemplos mal clasificados. En este caso se correspondería a la función de pérdida 0/1, que en el caso de probabilidades asignaría una pérdida de 0 a valores menores que 0.5 y 1 en caso contrario ¿Porqué no es una buena idea optimizar directamente esta función? Representa las dos funciones.

In [142]:
binary_loss_function = lambda Px: 0 if Px < 0.5 else 1

In [154]:
e_gradient_function = grad(binary_loss_function, argnums=1, allow_int=True)

In [ ]:
e_gradient_function(np.array([0,1]))